In [ ]:
import csv

with open('celtic_train.tsv', encoding="utf-8") as file:
  f = csv.reader(file, delimiter="\t")
  all_data = []
  for line in f:
    all_data.append(line)

In [ ]:
len(all_data)

5057059

In [ ]:
all_data[33]

['<S>', 'N']

In [ ]:
all_data = [['<S>', 'N']] + all_data

In [ ]:
def train_test_split(data, percent = 0.2):
    percent = int(percent * 10)

    train = data[((len(data) // 10) * percent):]
    test = data[:((len(data) // 10) * percent)]

    return train, test

In [ ]:
len(all_data)

5057060

In [56]:
all_data[0]

['<S>', 'N']

In [ ]:
train, test = train_test_split(all_data)

In [ ]:
len(train)

4045648

In [ ]:
len(test)

1011412

In [57]:
test[0]

['<S>', 'N']

In [ ]:
word_tag_count = dict()
N_count, S_count, U_count, T_count, H_count = 0, 0, 0, 0, 0 
unique_words = []
for word_tag in train:
  word, tag = word_tag[0], word_tag[1]
  
  if word == "<S>": continue
  
  if tag == 'N': N_count += 1
  elif tag == 'S': S_count += 1
  elif tag == 'T': T_count += 1
  elif tag == 'U': U_count += 1
  else: H_count += 1

  if word not in word_tag_count:
    word_tag_count[word] = {tag : 1}
    unique_words.append(word)
  else:
    if tag not in word_tag_count[word]: word_tag_count[word][tag] = 1
    else: word_tag_count[word][tag] += 1

In [ ]:
print(len(word_tag_count))
print(len(unique_words))

110712
110712


In [ ]:
print(N_count)
print(S_count)
print(T_count)
print(U_count)
print(H_count)

3312199
393888
14233
132859
32297


In [ ]:
total_count_per_tag = {'N' : N_count, 'S': S_count, 'T' : T_count, 'U': U_count, 'H' : H_count}
for word in unique_words:
  new_dict = {'N' : 0, 'S': 0, 'T' : 0, 'U': 0, 'H' : 0}
  tags = ['N', 'S', 'U', 'T', 'H']
  for tag in tags:
    if tag in word_tag_count[word]:
      new_dict[tag] = word_tag_count[word][tag] / total_count_per_tag[tag] 
  word_tag_count[word] = new_dict

In [ ]:
total_count_per_tag

{'N': 3312199, 'S': 393888, 'T': 14233, 'U': 132859, 'H': 32297}

In [ ]:
word_tag_count['tug']

{'N': 2.41531381417602e-06,
 'S': 0.005872227638313429,
 'T': 0,
 'U': 0.00027096395426730595,
 'H': 0}

In [60]:
sentence_context_tag_count = {'<S>' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}, 'N' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}, 
                              'S' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}, 'T' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0},
                              'U' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}, 'H' : {'N': 0, 'S': 0, 'T': 0, 'U': 0, 'H': 0}}

for i in range(1, len(train)):
  if train[i][0] == '<S>': continue
  word, tag = train[i][0], train[i][1]
  prev_word, prev_tag = train[i - 1][0], train[i - 1][1]
  if prev_word == '<S>': prev_tag = '<S>'
  sentence_context_tag_count[prev_tag][tag] += 1

In [61]:
sentence_context_tag_count

{'<S>': {'N': 144466, 'S': 15240, 'T': 12, 'U': 283, 'H': 170},
 'N': {'N': 2625779, 'S': 351222, 'T': 14180, 'U': 131993, 'H': 31298},
 'S': {'N': 370775, 'S': 20324, 'T': 24, 'U': 466, 'H': 620},
 'T': {'N': 13296, 'S': 834, 'T': 3, 'U': 2, 'H': 38},
 'U': {'N': 127090, 'S': 5048, 'T': 11, 'U': 88, 'H': 91},
 'H': {'N': 30792, 'S': 1220, 'T': 3, 'U': 27, 'H': 80}}

In [62]:
for tag in ['N', 'S', 'T', 'U', 'H', '<S>']:
  total_tag_sum = sum(sentence_context_tag_count[tag].values())
  for next_tag in ['N', 'S', 'T', 'U', 'H']:
    sentence_context_tag_count[tag][next_tag] /= total_tag_sum

In [63]:
sentence_context_tag_count

{'<S>': {'N': 0.9019485424952083,
  'S': 0.09514831024342733,
  'T': 7.491992932553334e-05,
  'U': 0.0017668616665938279,
  'H': 0.0010613656654450555},
 'N': {'N': 0.8323988927465515,
  'S': 0.11134097877552884,
  'T': 0.004495205536774459,
  'U': 0.04184313571336186,
  'H': 0.009921787227783287},
 'S': {'N': 0.9453505656422978,
  'S': 0.05181931062265272,
  'T': 6.119186454160919e-05,
  'U': 0.0011881420365162452,
  'H': 0.0015807898339915709},
 'T': {'N': 0.9381217808509137,
  'S': 0.0588442813800889,
  'T': 0.0002116700769067946,
  'U': 0.00014111338460452973,
  'H': 0.002681154307486065},
 'U': {'N': 0.9604165407170062,
  'S': 0.038147633153981016,
  'T': 8.31267758902122e-05,
  'U': 0.0006650142071216976,
  'H': 0.0006876851460008463},
 'H': {'N': 0.9585953552082684,
  'S': 0.037980200485648465,
  'T': 9.339393562044705e-05,
  'U': 0.0008405454205840234,
  'H': 0.002490504949878588}}